In [ ]:
!pip install segmentation_models_pytorch

# Iniatial

In [ ]:
!mkdir Master
!mkdir /content/Master/instances
!mkdir /content/Master/Output

# !wget -P Master http://images.cocodataset.org/zips/val2017.zip
!wget -P Master http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
!wget -P /content/Master/instances http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget -P Master http://images.cocodataset.org/zips/train2017.zip

--2023-06-04 07:23:33--  http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.8.151, 52.217.195.129, 3.5.29.133, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.8.151|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 860725834 (821M) [application/zip]
Saving to: ‘Master/panoptic_annotations_trainval2017.zip’

panoptic_annotation 100%[===================>] 820.85M  95.5MB/s    in 8.5s    

2023-06-04 07:23:41 (96.1 MB/s) - ‘Master/panoptic_annotations_trainval2017.zip’ saved [860725834/860725834]

--2023-06-04 07:23:41--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.140.233, 52.217.105.116, 52.216.138.179, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.140.233|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25

In [ ]:
# !unzip /content/Master/val2017.zip -d Master
!unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master
!unzip /content/Master/instances/annotations_trainval2017.zip -d /content/Master/instances
!unzip /content/Master/train2017.zip -d Master

Streaming output truncated to the last 5000 lines.
 extracting: Master/train2017/000000103581.jpg  
 extracting: Master/train2017/000000004608.jpg  
 extracting: Master/train2017/000000529532.jpg  
 extracting: Master/train2017/000000023311.jpg  
 extracting: Master/train2017/000000048905.jpg  
 extracting: Master/train2017/000000570263.jpg  
 extracting: Master/train2017/000000120666.jpg  
 extracting: Master/train2017/000000548471.jpg  
 extracting: Master/train2017/000000121965.jpg  
 extracting: Master/train2017/000000524535.jpg  
 extracting: Master/train2017/000000350675.jpg  
 extracting: Master/train2017/000000537456.jpg  
 extracting: Master/train2017/000000470613.jpg  
 extracting: Master/train2017/000000580837.jpg  
 extracting: Master/train2017/000000050410.jpg  
 extracting: Master/train2017/000000255158.jpg  
 extracting: Master/train2017/000000500356.jpg  
 extracting: Master/train2017/000000241911.jpg  
 extracting: Master/train2017/000000181990.jpg  
 extracting: Maste

In [ ]:
# !unzip /content/Master/annotations/panoptic_val2017.zip -d Master
# !mv /content/Master/annotations/panoptic_val2017.json /content/Master # move src dest
# !mv /content/Master/instances/annotations/instances_val2017.json /content/Master # move src dest
# # !unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master

!rm -r /content/Master/panoptic_annotations_trainval2017.zip
!rm -r /content/Master/train2017.zip

!unzip /content/Master/annotations/panoptic_train2017.zip -d Master
!mv /content/Master/annotations/panoptic_train2017.json /content/Master # move src dest
!mv /content/Master/instances/annotations/instances_train2017.json /content/Master # move src dest
# !unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master

In [ ]:
!rm -r /content/Master/val2017.zip
# !rm -r /content/Master/panoptic_annotations_trainval2017.zip
# !rm -r /content/Master/train2017.zip

import shutil

shutil.rmtree('/content/Master/annotations', ignore_errors=True)
shutil.rmtree('/content/Master/instances', ignore_errors=True)
shutil.rmtree('/content/Master/__MACOSX', ignore_errors=True)

In [ ]:
# import shutil

# shutil.rmtree('/content/Proiect-Dizertatie', ignore_errors=True)
# shutil.rmtree('/content/Master/instances', ignore_errors=True)

# Libraries

In [ ]:
# For managing COCO dataset
from pycocotools.coco import COCO

# For creating and managing folder/ files
import glob
import os
import shutil

# For managing images
from PIL import Image
import skimage.io as io

# Basic libraries
import numpy as np
import pandas as pd
import random
import cv2

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

# For importing models and working with them
## Torch
import torch
import torch.utils.data # for Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp

## Torchvision
import torchvision
from torchvision.transforms import transforms

# For creating train - test splits
from sklearn.model_selection import train_test_split

import pathlib
import pylab
import requests
from io import BytesIO
from pprint import pprint
from tqdm import tqdm
import time
from imutils import paths

# Performance Metrics
from sklearn.metrics import multilabel_confusion_matrix


# Import Files
import BinaryMasks as bm
import CalcMetrics as cm
import CreateDataset as crd
import FilterImgs as flt
import TrainModel as trModel
import ValidateModel as valModel
import TestModel as testModel
import SaveCSV
import DiceLoss as dcloss
import EarlyStopping as stopping
import PlotResults as pr
import MainLoop as main
import DisplayMetrics as dm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# %matplotlib inline

# Configuration

In [ ]:
data = 'train2017'

# Data and Masks
IMAGE_PATH = os.path.join('/content/Master/', data)
MASK_PATH = os.path.join('/content/Master/', 'panoptic_' + data)

# define the path to the base output directory
BASE_OUTPUT = '/content/Master/Output'

# define the path to the output serialized model, model training plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_tgs_salt.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])



# determine if we will be pinning memory during data loading
PIN_MEMORY = True if device == "cuda" else False


# define the number of channels in the input, number of classes, and number of levels in the U-Net model
# NUM_CHANNELS = 3
# NUM_CLASSES = 2 # person vs Background

# define the input image dimensions
INPUT_IMAGE_WIDTH = 256
INPUT_IMAGE_HEIGHT = 256

# define threshold to filter weak predictions
THRESHOLD = 0.5 # for binary classification

# Data

## Data Preparation

In [ ]:
# Prepare Directory for Binary Masks
shutil.rmtree('/content/Master/Binary_Masks', ignore_errors = True)

!mkdir /content/Master/Binary_Masks

ann_path = '/content/Master/instances_train2017.json'
filterCls = ['person']
bm_path = '/content/Master/Binary_Masks/'

small = 9000
medium = 15000
large = 30000

imagePaths, maskPaths, binaryMaskPaths = flt.filter_data(ann_path, filterCls, bm_path, IMAGE_PATH, MASK_PATH)
SaveCSV.create_csvs(imagePaths, binaryMaskPaths, small, medium, large)

In [ ]:
# IMG_PATHS = os.path.sep.join([BASE_OUTPUT, "img_paths.txt"])
# BMASKS_PATHS = os.path.sep.join([BASE_OUTPUT, "bmasks_paths.txt"])

# print("[INFO] saving image paths...")
# f = open(IMG_PATHS, "w")
# f.write("\n".join(imagePaths))
# f.close()

# print("[INFO] saving binary masks paths...")
# f = open(BMASKS_PATHS, "w")
# f.write("\n".join(binaryMaskPaths))
# f.close()

[INFO] saving image paths...
[INFO] saving binary masks paths...


## Train

In [ ]:
# https://github.com/qubvel/segmentation_models.pytorch
# https://github.com/qubvel/segmentation_models.pytorch#architectures

RESNET_SIZE = 256
transf_size = RESNET_SIZE

# Initialize transformations - train
transforms_train = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize transformations - test (optional)
transforms_test = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])


# Define hyperparameters
INIT_LR = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = [10, 20, 50, 100, 150]
TEST_SPLIT = [0.1, 0.2, 0.3] # 0.3
VALID_SPLIT = 0.1 # 0.1

wb_name = "Resnet_Unet_MSCOCO"
loaded_df = pd.read_csv(r'shuffled_df_medium.csv')

main.main_loop(loaded_df, TEST_PATHS, transforms_train, transforms_test, wb_name, INIT_LR, BATCH_SIZE, NUM_EPOCHS, TEST_SPLIT, VALID_SPLIT, PIN_MEMORY)



